In [0]:
#  Connection 1  
pg_host0   = "ep-misty-field-aeg07ur8.c-2.us-east-2.aws.neon.tech"
pg_port0   = "5432"
pg_db0     = "neondb"
pg_schema0 = "public"
pg_table0  = "referrals"                
pg_user0   = "neondb_owner"
pg_pwd0    = "npg_ZRuSiM2IBn7p"
sslmode0   = "require"

jdbc_url0  = f"jdbc:postgresql://{pg_host0}:{pg_port0}/{pg_db0}?sslmode={sslmode0}"
full_tbl0  = f'"{pg_schema0}"."{pg_table0}"'
jdbc_opts0 = {"url": jdbc_url0, "user": pg_user0, "password": pg_pwd0, "driver": "org.postgresql.Driver"}

# #--------------------------------------------------------------
 
#  Connection  2
pg_host1   = "ep-shy-rice-adbb0t2f.c-2.us-east-1.aws.neon.tech"
pg_port1   = "5432"
pg_db1     = "neondb"
pg_schema1 = "public"
pg_table1  = "referrals"                
pg_user1   = "neondb_owner"
pg_pwd1    = "npg_JAwf5danYy8q"
sslmode1   = "require"

jdbc_url1  = f"jdbc:postgresql://{pg_host1}:{pg_port1}/{pg_db1}?sslmode={sslmode1}"
full_tbl1  = f'"{pg_schema1}"."{pg_table1}"'
jdbc_opts1 = {"url": jdbc_url1, "user": pg_user1, "password": pg_pwd1, "driver": "org.postgresql.Driver"}

#-----------------------------------------------------------------------------------------------------
# Connection  3
pg_host2   = "ep-bitter-cake-adhdrtjy.c-2.us-east-1.aws.neon.tech"
pg_port2   = "5432"
pg_db2     = "neondb"
pg_schema2 = "public"
pg_table2  = "referrals"                
pg_user2   = "neondb_owner"
pg_pwd2    = "npg_meDT9zHBi6hl"
sslmode2   = "require"

jdbc_url2  = f"jdbc:postgresql://{pg_host2}:{pg_port2}/{pg_db2}?sslmode={sslmode2}"
full_tbl2  = f'"{pg_schema2}"."{pg_table2}"'
jdbc_opts2 = {"url": jdbc_url2, "user": pg_user2, "password": pg_pwd2, "driver": "org.postgresql.Driver"}

df0 = (spark.read
      .format("jdbc")
      .options(**jdbc_opts0)
      .option("dbtable", full_tbl0)
      .load())
df1 = (spark.read
      .format("jdbc")
      .options(**jdbc_opts1)
      .option("dbtable", full_tbl1)
      .load())
df2 = (spark.read
      .format("jdbc")
      .options(**jdbc_opts2)
      .option("dbtable", full_tbl2)
      .load())
# df0=df0.filter(df0.sent == False)     
# df1=df1.filter(df1.sent == False)
# df2=df2.filter(df2.sent == False)



In [0]:
%sql
truncate table bridge_dev.bridge_connect.referrals;

In [0]:
from pyspark.sql.functions import lit
df_db0_tag= df0.withColumn("source_db", lit("db0")).withColumn("company_email", lit("general@bridgeconnect.azimut"))
df_db1_tag = df1.withColumn("other_fields", lit(None).cast("string")).withColumn("source_db", lit("db1")).withColumn("company_email", lit("general@bridgeconnect.azimut"))
df_db2_tag = df2.withColumn("other_fields", lit(None).cast("string")).withColumn("source_db", lit("db2")).withColumn("company_email", lit("general@bridgeconnect.azimut"))



### to create the table for first time

In [0]:
# df_db0_tag.createOrReplaceTempView("referrals")

In [0]:
# %sql
# CREATE OR REPLACE TABLE bridge_dev.bridge_connect.referrals  
# USING DELTA
# AS
# SELECT
#   r.*
# FROM referrals r;

In [0]:
table = "bridge_dev.bridge_connect.referrals"
df_db0_tag.write.format("delta").mode("append").saveAsTable(table)
df_db1_tag.write.format("delta").mode("append").saveAsTable(table)
df_db2_tag.write.format("delta").mode("append").saveAsTable(table)

In [0]:

df = spark.table(table)
df_clean = df.dropDuplicates()
df_clean.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable(table)



